In [1]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
import tempfile

In [2]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [3]:
tempdir = tempfile.TemporaryDirectory()
application = Application(client, compiler, "templates", tempdir.name)

In [4]:
application_description = """
Bot that helps me planning my YouTube content and posting schedule.
""".strip()

In [5]:
my_bot = application.create_bot(application_description)

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


Compiling TypeSpec...
Compiling Typescript Schema Definitions...
Compiling Drizzle...
Generating Router...
Compiling Handlers...
Generating Application...


In [10]:
my_bot

ApplicationOut(typespec=TypespecOut(reasoning='For a hello bot that greets on every message, we need:\n1. A greeting function that takes user\'s message and generates appropriate response\n2. The greeting should consider user\'s name if provided in the message\n3. Time of day can be extracted to provide appropriate greeting (good morning/afternoon/evening)\n4. Should track conversation history to avoid repetitive greetings\n\nExample interactions:\n"Hi" -> greet({timeOfDay: "morning"})\n"Hello, I\'m John" -> greet({userName: "John", timeOfDay: "evening"})', typespec_definitions='model GreetingContext {\n    userName?: string;\n    timeOfDay: string;\n}\n\ninterface HelloBot {\n    @llm_func(2)\n    greet(context: GreetingContext): string;\n}', llm_functions=['greet'], error_output=None), drizzle=DrizzleOut(reasoning='The TypeSpec shows:\n1. A GreetingContext model with userName (optional) and timeOfDay fields\n2. A HelloBot interface that processes greetings\n3. We need to store both t

In [11]:
application.generation_dir

'/tmp/tmpka_uy94u/generated/generation-1'

In [8]:
!pushd {application.generation_dir}/app_schema
!npm install
!npx tsc src/main.ts
!popd


/tmp/tmpka_uy94u/generated/generation-1/app_schema ~/neon/bot-new-1/agent


npm ERR! code ENOENT
npm ERR! syscall open
npm ERR! path /home/eugenek/neon/bot-new-1/agent/package.json
npm ERR! errno -2
npm ERR! enoent ENOENT: no such file or directory, open '/home/eugenek/neon/bot-new-1/agent/package.json'
npm ERR! enoent This is related to npm not being able to find a file.
npm ERR! enoent 

npm ERR! A complete log of this run can be found in:
npm ERR!     /home/eugenek/.npm/_logs/2025-02-07T11_44_06_379Z-debug-0.log

                                                                               
                This is not the tsc command you are looking for                
                                                                               

To get access to the TypeScript compiler, tsc, from the command line either:

- Use npm install typescript to first add TypeScript to your project before using npx
- Use yarn to avoid accidentally running code from un-installed packages
/bin/bash: line 1: popd: directory stack empty


In [9]:
typespec = application._make_typespec(application_description)
typespec.score, typespec.data.output

AttributeError: 'TypespecOut' object has no attribute 'score'

In [ ]:
typescript_schema = application._make_typescript_schema(typespec.data.output.typespec_definitions)
typescript_schema.score, typescript_schema.data.output

In [ ]:
drizzle = application._make_drizzle(typespec.data.output.typespec_definitions)
drizzle.score, drizzle.data.output

In [ ]:
router = application._make_router(typespec.data.output.typespec_definitions)
router.score, router.data.output

In [ ]:
handlers = application._make_handlers(
    typespec.data.output.llm_functions,
    typespec.data.output.typespec_definitions,
    typescript_schema.data.output.typescript_schema,
    drizzle.data.output.drizzle_schema,
)

In [ ]:
for name, handler in handlers.items():
    print(name, handler.score, handler.depth, handler.data.output)
    if handler.score != 1:
        print(handler.data.output.feedback["stdout"])

recordExercise 1 0 HandlerOutput(handler='import { db } from "../db";\nimport type { Exercise } from "../common/schema";\nimport { exerciseTable } from "../db/schema/application";\n\ninterface Options {\n    exercise: Exercise;\n}\n\nexport const handle = async (options: Options): Promise<void> => {\n    await db.insert(exerciseTable).values({\n        name: options.exercise.name,\n        sets: options.exercise.sets,\n        reps: options.exercise.reps,\n        weight: options.exercise.weight,\n        notes: options.exercise.notes\n    }).execute();\n};', feedback={'exit_code': 0, 'stdout': None, 'stderr': None})
updateAvailableEquipment 1 0 HandlerOutput(handler='import { db } from "../db";\nimport { equipmentTable } from "../db/schema/application";\nimport type { Equipment } from "../common/schema";\n\ninterface Options {\n    equipment: Equipment[];\n}\n\nexport const handle = async (options: Options): Promise<void> => {\n    // Clear existing equipment\n    await db.delete(equi